## importing python libraries

In [8]:
!pip freeze> requirements.txt

In [15]:
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import face_recognition
import keras
from keras.models import load_model
import cv2


In [16]:
#GET_EMOTION BY FRAME
import cv2
import numpy as np

emotion_dict= {'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}

def get_emotion(frame, show_predictions = False):
    
    face_image = frame

    # resizing the image
    face_image = cv2.resize(face_image, (48,48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
    
    #load the model trained for detecting emotions of a face
    model = load_model("../emotion_detector_models/model_v6_23.hdf5")

    predicted_class = np.argmax(model.predict(face_image))
    #prediction
    label_map = dict((v,k) for k,v in emotion_dict.items()) 
    predicted_label = label_map[predicted_class] if predicted_class not in (2,1)  else label_map[4]
    if show_predictions:
        print("pred",predicted_label)
    label = "unhappy" if predicted_label in ('Angry', 'Fear', 'Sad', 'Disgust') else ('happy' if predicted_label in ('Surprise', "Happy") else 'neutral')
    return label



In [17]:
#GO THROUGH VIDEO FRAME BY FRAME TO GET EMOTIONS
#take every 5th frame and check the emotion

# Importing all necessary libraries for this function
import cv2
import os

#IF YOU WANT TO DIRECTLY RUN FUNCTION IN THIS CELL
# Read the video from specified path
# video_name = "toddler"
# full_video_name = video_name + ".mov"
# video_input_path = "../input/"+full_video_name
# video_output_path = "../output/"+video_name
# cam = cv2.VideoCapture(video_input_path)
# window_size = 5
    
def get_frames_and_emotion(video_name, video_output_path, sample_rate, show_predictions = False):
    try:

        # creating a folder named data
        if not os.path.exists(video_output_path):
            os.makedirs(video_output_path)

    # if not created then raise error
    except OSError:
        print('Error: Creating directory of data')
    
    # frame
    currentframe = 0

    emotionByFrames = []
    
    while(True):

        # reading from frame
        ret,frame = cam.read()

        if ret:
            # if video is still left continue creating images
            name = '../output/'+video_name+'/frame' + str(currentframe) + '.jpg'
    #         print ('Creating...' + name)
            if currentframe%sample_rate ==0:
                emotionByFrames.append(get_emotion(frame, show_predictions))
            # writing the extracted images
            cv2.imwrite(name, frame)

            # increasing counter so that it will
            # show how many frames are created
            currentframe += 1

        else:
            cam.release()
            cv2.destroyAllWindows()
            break

        # Release all space and windows once done 
    return emotionByFrames, currentframe

#If you want to run this function directly in this cell
# emotionByFrames, totalFrames = get_frames_and_emotion(video_name, video_output_path, window_size)
# print(emotionByFrames, totalFrames)  

In [18]:
import csv
import numpy as np
import cv2
import os
from collections import Counter

def smooth_moving_data(emotionByFrames, sample_rate, window_size): #smooth emotions so that the bounding box is more consistent
    smoothed_list = []
    for i, emotion in enumerate(emotionByFrames): #check emotions in a sliding window and add the most frequent
        occurence_count = Counter(emotionByFrames[i:i+window_size])
        most_frequent_emotion = [occurence_count.most_common(1)[0][0]]
        smoothed_list.extend(most_frequent_emotion*sample_rate) #populate the  smoothed list so there is an emotion for every timeframe
    return smoothed_list
    
# print("EMOTION BY FRAMES", emotionByFrames, len(emotionByFrames))
# smoothed_list = smooth_moving_data(emotionByFrames, window_size = 5)
# print("OUTPUT", smoothed_list, len(smoothed_list))



In [13]:
def create_csv(smoothed_list, video_output_path):
    
    # Open a new CSV file in write mode
    with open(video_output_path+'.csv', 'w', newline='') as file:
        # Create a CSV writer object
        writer = csv.writer(file)
        writer.writerow(["index","emotions"])
        for i, emotion in enumerate(smoothed_list, start=1):
            writer.writerow([i, emotion])
    return True

# result = create_csv(smoothed_list, window_size = 5)
# print(result)

## Run all scripts together

In [14]:
# Read the video from specified path
video_name = "nameOfVideoWithoutVideoExtension" #without video format ex. "infant2"
full_video_name = video_name + "videoExtension" #fill in video format ex. ".mov"
video_input_path = "../../input/"+full_video_name
video_output_path = "../output/"+video_name
cam = cv2.VideoCapture(video_input_path)
sample_rate = 5
window_size = 5

print("OS", os.getcwd()) #GET COMPUTER PATH

#first go through each frame and detect emotions every window_size frames
emotionByFrames, totalFrames = get_frames_and_emotion(video_name, video_output_path, sample_rate, show_predictions = False)

#go through the emotions and smooth data
smoothed_list = smooth_moving_data(emotionByFrames, sample_rate, window_size)

#create a csv with the emotions at a given timeframe
result = create_csv(smoothed_list, video_output_path)


OS /Users/arielf/Desktop/6.8510_final_project/face_and_emotion_detection/src
1/1 [==============================] - 0s 73ms/step


1/1 [==============================] - 0s 67ms/step
